In [4]:
# 파일 경로 설정
file_path = '/Users/dryoon04/Documents/GitHub/schoolproject/vectorstore/open ai api key.txt'

# 파일 열기 (읽기 모드로 열기)
with open(file_path, 'r', encoding='utf-8') as file:
    # 파일 내용 읽어오기
    file_content = file.read()

In [5]:
import pandas as pd
import chromadb
import os
import openai


In [6]:
df = pd.read_csv('lyrics.csv')

In [7]:
client = chromadb.PersistentClient("chromadb/lyrics.db")
collection = client.get_or_create_collection(name="lyrics",metadata={"hnsw:space":"cosine"})


In [8]:
singers = []
song_titles = []
lyrics = []
ids = []
documents = []

for idx in range(len(df)):
    item = df.iloc[idx]
    singer = item['가수'].strip().lower()  # 가수 정보
    song_title = item['제목'].strip().lower()  # 노래 제목 정보
    lyric = item['가사'].strip().lower()  # 가사 정보

    # id = f"{singer}-{song_title}".replace(' ', '-')  # 고유한 ID 생성
    id = f"{singer}-{song_title}"
    document = f"{singer}:{song_title}:{lyric}"  # 문서 텍스트 생성
    
    
    ids.append(id)
    documents.append(document)

# DB 저장
collection.add(documents=documents,ids=ids)


Add of existing embedding ID: stayc(스테이씨)-bubble
Add of existing embedding ID: 영탁-폼미쳤다
Add of existing embedding ID: 부석순(seventeen)-파이팅해야지(feat.이영지)
Add of existing embedding ID: charlieputh-that'shilarious
Add of existing embedding ID: 성시경-너의모든순간
Add of existing embedding ID: nctdream-yogurtshake
Add of existing embedding ID: 테이-monologue
Add of existing embedding ID: charlieputh-that’snothowthisworks(feat.dan+shay)
Add of existing embedding ID: 임영웅-이제나만믿어요
Add of existing embedding ID: 김호중-인생은뷰티풀
Add of existing embedding ID: 임영웅-우리들의블루스
Add of existing embedding ID: 임영웅-abientot
Add of existing embedding ID: 임영웅-인생찬가
Add of existing embedding ID: 오마이걸(ohmygirl)-여름이들려(summercomes)
Add of existing embedding ID: ive(아이브)-eitherway
Add of existing embedding ID: 방탄소년단-butter
Add of existing embedding ID: 정국-stillwithyou
Add of existing embedding ID: v-slowdancing
Add of existing embedding ID: 임영웅-보

In [9]:
user_query = input("검색어를 입력하세요: ")
vector_res = collection.query(query_texts=[f"{user_query}"],n_results=5)
print(user_query)
print(vector_res)


{'ids': [['charlieputh-that’snothowthisworks(feat.dan+shay)', '지민-likecrazy', "charlieputh-that'shilarious", 'newjeans-asap', 'nctu-baggyjeans']], 'distances': [[0.8533503413200378, 0.8709683418273926, 0.8768119812011719, 0.8802019357681274, 0.8834594488143921]], 'metadatas': [[None, None, None, None, None]], 'embeddings': None, 'documents': [['charlieputh:that’snothowthisworks(feat.dan+shay):thought the day you disappeared\nthat it was over\ndidn\'t even hear you leavin\'\nsaw you with someone and\nthought that it was closure\nbut you still tell me\nthat you need me\nbaby, why, tell me, why\ncan\'t you just make up your mind?\nquit messin\' with my head\nyou can\'t say you hate me,\nthen call me when you\'re hurt\nbaby, you know that\'s not\nhow this works, no\nthat\'s not how this works\nyou can\'t walk away,\nthen come back to what we were\nbaby, you know that\'s not\nhow this works, no\nthat\'s not how this works, no\nno, no, no, no\noh, no, no, no, no\nbaby, that\'s not how th

In [10]:

os.environ["OPENAI_API_KEY"] = file_content
openai.api_key = os.getenv("OPENAI_API_KEY")
user_query = input("검색어를 입력하세요: ")
vector_res = collection.query(query_texts=[f"{user_query}"],n_results=5)
print(vector_res)
srchres = []
for v in vector_res['documents'][0]:
    item = v.split(':')
    srchres.append({"가수" : item[0].strip(),"제목" : item[1].strip(),"가사" : item[2].strip()})
    
completion = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=[
               {"role": "system","content": "You are an assistant who can answer questions about music lyricsBased on the following database In order to answer questions, you should only use facts from the databaseIf you feel you don't have enough information to answer a question, you should say, '답할수있는 정보가 없습니다.'."},
            {"role": "user", "content": f"{user_query}"},
            {"role": "assistant", "content": f"{srchres}"},
        ],
        temperature=1,
        max_tokens=256,
        top_p=1,
        frequency_penalty=0,
        presence_penalty=0,
    )
    
print(completion["choices"][0]["message"]["content"])


{'ids': [['newjeans-newjeans', 'newjeans-coolwithyou', 'newjeans-attention', 'newjeans-ditto', 'newjeans-eta']], 'distances': [[0.500866174697876, 0.526356041431427, 0.5886397361755371, 0.5954025983810425, 0.6027153730392456]], 'metadatas': [[None, None, None, None, None]], 'embeddings': None, 'documents': [["newjeans:newjeans:look it's a new me\nswitched it up, who's this? \n우릴 봐 newjeans \nso fresh, so clean\n얼마나\n기다렸던 날\n드디어\ntime to step out \n또 한 번 더\nready for sure \nto have some more\nnew hair \nnew tee\nnewjeans \ndo you see\nnew hair \nnew tee\nnewjeans \ndo you see\nnew hair \nnew tee\nnewjeans \ndo you see\nnew hair \nnew tee\nnewjeans \ndo you see\nmake it feel\nlike a game\nlook at us, we go\non & on again\nwe'll go on to the end \nwhat we wanna do\non & on again\nmake it feel\nlike a game\nlook at us, we go\non & on again\nwe'll go on to the end \nwhat we wanna do\non & on again\nlook it's a new me \nswitched it up, who's this? \n들어봐 newjeans \nso fresh, so clean\n얼마나\n기다

In [11]:
user_query = input("검색어를 입력하세요: ")
vector_res = collection.query(query_texts=[f"{user_query}"],n_results=10)
print(user_query)
print(vector_res)


{'ids': [['charlieputh-that’snothowthisworks(feat.dan+shay)', '지민-likecrazy', "charlieputh-that'shilarious", 'newjeans-asap', 'nctu-baggyjeans', 'lesserafim(르세라핌)-unforgiven(feat.nilerodgers)', 'fiftyfifty-cupid', '정국,방탄소년단-dreamers[musicfromthefifaworldcupqatar2022officialsoundtrack](feat.fifasound)', 'newjeans-newjeans', '방탄소년단-butter']], 'distances': [[0.8533503413200378, 0.8709683418273926, 0.8768119812011719, 0.8802019357681274, 0.8834594488143921, 0.8905946016311646, 0.8917070031166077, 0.8919647192552597, 0.8951927423477173, 0.8956776261329651]], 'metadatas': [[None, None, None, None, None, None, None, None, None, None]], 'embeddings': None, 'documents': [['charlieputh:that’snothowthisworks(feat.dan+shay):thought the day you disappeared\nthat it was over\ndidn\'t even hear you leavin\'\nsaw you with someone and\nthought that it was closure\nbut you still tell me\nthat you need me\nbaby, why, tell me, why\ncan\'t you just make up your mind?\nquit me